In [1]:
from functools import partial
from collections import Counter
import numpy as np
import pandas as pd
import optuna
from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
import tensorflow as tf
import keras_tuner as kt

/home/nicomore/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-22 19:24:53.678358: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-22 19:24:53.827068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-22 19:24:53.879620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-22 19:

In [2]:
df = pd.read_csv('train.csv').sample(frac=1)
df_sub = pd.read_csv('test.csv').sample(frac=1)

In [3]:
df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
7427,7427,2.0,No,4.0,5.0,No,15.0,5.0,Extrovert
1239,1239,0.0,No,4.0,5.0,No,9.0,10.0,Extrovert
13783,13783,3.0,No,9.0,7.0,No,13.0,7.0,Extrovert
15054,15054,1.0,No,5.0,4.0,No,NaN,8.0,Extrovert
16405,16405,1.0,No,4.0,3.0,No,12.0,10.0,Extrovert


In [4]:
df['Stage_fear'] = np.where(df['Stage_fear'] == 'Yes', 1, 0)
df['Drained_after_socializing'] = np.where(df['Drained_after_socializing'] == 'Yes', 1, 0)
df['Personality'] = np.where(df['Personality'] == 'Introvert', 1, 0)

df_sub['Stage_fear'] = np.where(df_sub['Stage_fear'] == 'Yes', 1, 0)
df_sub['Drained_after_socializing'] = np.where(df_sub['Drained_after_socializing'] == 'Yes', 1, 0)

In [5]:
df['Reclusion'] = df['Time_spent_Alone'] - df['Going_outside']
df['Social_Drainage'] = df['Drained_after_socializing'] / (df['Friends_circle_size'] + 1)
df['Actual_Fear_Exposure'] = df['Stage_fear'] / (df['Post_frequency'] + 1)
df['Media_Dependence'] = df['Post_frequency'] / (df['Time_spent_Alone'] + 1)
df['Social_Volume'] = df['Social_event_attendance'] * df['Friends_circle_size'] 

In [6]:
df_sub['Reclusion'] = df_sub['Time_spent_Alone'] - df_sub['Going_outside']
df_sub['Social_Drainage'] = df_sub['Drained_after_socializing'] / (df_sub['Friends_circle_size'] + 1)
df_sub['Actual_Fear_Exposure'] = df_sub['Stage_fear'] / (df_sub['Post_frequency'] + 1)
df_sub['Media_Dependence'] = df_sub['Post_frequency'] / (df_sub['Time_spent_Alone'] + 1)
df_sub['Social_Volume'] = df_sub['Social_event_attendance'] * df_sub['Friends_circle_size'] 

In [7]:
imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=100, random_state=47, n_jobs=-1), max_iter=50, random_state=47)
personality = df['Personality']

imputed_array = imputer.fit_transform(df.drop('Personality', axis=1))
imputed_array_sub = imputer.transform(df_sub)

In [8]:
df = pd.DataFrame(imputed_array, columns=df.drop('Personality', axis=1).columns, index=df.drop('Personality', axis=1).index).drop('id', axis=1)
df['Personality'] = personality

df_sub = pd.DataFrame(imputed_array_sub, columns=df_sub.columns, index=df_sub.index)

In [9]:
usedcols = []
cols = df.drop('Personality', axis=1).columns

for col in cols:
    usedcols.append(col)
    for col2 in cols:
        if col != col2:
            df[col + '_/_' + col2] = df[col] / (df[col2] + 1)
            df[col + '_-_' + col2] = df[col] - df[col2]
            df[col + '_**_' + col2] = df[col] ** df[col2]

        if col2 in usedcols:
            continue
        
        df[col + '_*_' + col2] = df[col] * df[col2]
        df[col + '_+_' + col2] = df[col] + df[col2]

        df['Log1p_' + col] = np.log1p(df[col])
        df['Sqrt_' + col] = np.sqrt(df[col])
        df['Exp_' + col] = np.exp(df[col])
        df['Square_' + col] = np.power(df[col], 2)
        df['Abs_' + col] = np.abs(df[col])

        df['Mean_Diff_' + col] = df[col] - df[col].mean()
        df['Zscore_' + col] = (df[col] - df[col].mean()) / (df[col].std() + 1e-6)
        df['Minmax_' + col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min() + 1e-6)
        df['Rank_' + col] = df[col].rank(method='average')
        df['Quantile_' + col] = pd.qcut(df[col].rank(method='average'), q=10, labels=False, duplicates='drop')

        df['GT_Mean_' + col] = (df[col] > df[col].mean()).astype(int)
        df['GT_Median_' + col] = (df[col] > df[col].median()).astype(int)

/tmp/ipykernel_108473/877111518.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col + '_-_' + col2] = df[col] - df[col2]
/tmp/ipykernel_108473/877111518.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col + '_**_' + col2] = df[col] ** df[col2]
/tmp/ipykernel_108473/877111518.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmen

In [10]:
usedcols = []
for col in cols:
    usedcols.append(col)
    for col2 in cols:
        if col != col2:
            df_sub[col + '_/_' + col2] = df_sub[col] / (df_sub[col2] + 1)
            df_sub[col + '_-_' + col2] = df_sub[col] - df_sub[col2]
            df_sub[col + '_**_' + col2] = df_sub[col] ** df_sub[col2]

        if col2 in usedcols:
            continue
        
        df_sub[col + '_*_' + col2] = df_sub[col] * df_sub[col2]
        df_sub[col + '_+_' + col2] = df_sub[col] + df_sub[col2]

        df_sub['Log1p_' + col] = np.log1p(df_sub[col])
        df_sub['Sqrt_' + col] = np.sqrt(df_sub[col])
        df_sub['Exp_' + col] = np.exp(df_sub[col])
        df_sub['Square_' + col] = np.power(df_sub[col], 2)
        df_sub['Abs_' + col] = np.abs(df_sub[col])

        df_sub['Mean_Diff_' + col] = df_sub[col] - df_sub[col].mean()
        df_sub['Zscore_' + col] = (df_sub[col] - df[col].mean()) / (df[col].std() + 1e-6)
        df_sub['Minmax_' + col] = (df_sub[col] - df[col].min()) / (df[col].max() - df[col].min() + 1e-6)
        df_sub['Rank_' + col] = df_sub[col].rank(method='average')
        df_sub['Quantile_' + col] = pd.qcut(df_sub[col].rank(method='average'), q=10, labels=False, duplicates='drop')

        df_sub['GT_Mean_' + col] = (df_sub[col] > df[col].mean()).astype(int)
        df_sub['GT_Median_' + col] = (df_sub[col] > df[col].median()).astype(int)

/tmp/ipykernel_108473/1965970290.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_sub[col + '_**_' + col2] = df_sub[col] ** df_sub[col2]
/tmp/ipykernel_108473/1965970290.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_sub[col + '_*_' + col2] = df_sub[col] * df_sub[col2]
/tmp/ipykernel_108473/1965970290.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

In [11]:
df.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Reclusion,Social_Drainage,Actual_Fear_Exposure,...,Social_Volume_**_Reclusion,Social_Volume_/_Social_Drainage,Social_Volume_-_Social_Drainage,Social_Volume_**_Social_Drainage,Social_Volume_/_Actual_Fear_Exposure,Social_Volume_-_Actual_Fear_Exposure,Social_Volume_**_Actual_Fear_Exposure,Social_Volume_/_Media_Dependence,Social_Volume_-_Media_Dependence,Social_Volume_**_Media_Dependence
7427,2.0,0.0,4.0,5.0,0.0,15.0,5.0,-3.0,0.0,0.0,...,4.629630e-06,60.0,60.0,1.0,60.0,60.0,1.0,22.500000,58.333333,9.195713e+02
1239,0.0,0.0,4.0,5.0,0.0,9.0,10.0,-5.0,0.0,0.0,...,1.653817e-08,36.0,36.0,1.0,36.0,36.0,1.0,3.272727,26.000000,3.656158e+15
13783,3.0,0.0,9.0,7.0,0.0,13.0,7.0,-4.0,0.0,0.0,...,5.336500e-09,117.0,117.0,1.0,117.0,117.0,1.0,42.545455,115.250000,4.162222e+03
15054,1.0,0.0,5.0,4.0,0.0,10.0,8.0,-3.0,0.0,0.0,...,8.000000e-06,50.0,50.0,1.0,50.0,50.0,1.0,10.000000,46.000000,6.250000e+06
16405,1.0,0.0,4.0,3.0,0.0,12.0,10.0,-2.0,0.0,0.0,...,4.340278e-04,48.0,48.0,1.0,48.0,48.0,1.0,8.000000,43.000000,2.548040e+08


In [12]:
df = df.replace([np.inf, -np.inf], np.nan).fillna(0)

In [13]:
corr_matrix = df.drop('Personality', axis=1).corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

df = df.drop(to_drop, axis=1)

In [14]:
df_sub = df_sub.drop(to_drop, axis=1)

In [15]:
df.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Reclusion,Social_Drainage,Actual_Fear_Exposure,...,Social_Volume_**_Time_spent_Alone,Social_Volume_/_Going_outside,Social_Volume_**_Going_outside,Social_Volume_/_Post_frequency,Social_Volume_**_Post_frequency,Social_Volume_**_Reclusion,Social_Volume_**_Social_Drainage,Social_Volume_**_Actual_Fear_Exposure,Social_Volume_/_Media_Dependence,Social_Volume_**_Media_Dependence
7427,2.0,0.0,4.0,5.0,0.0,15.0,5.0,-3.0,0.0,0.0,...,3600.0,10.000,7.776000e+08,10.000000,7.776000e+08,4.629630e-06,1.0,1.0,22.500000,9.195713e+02
1239,0.0,0.0,4.0,5.0,0.0,9.0,10.0,-5.0,0.0,0.0,...,1.0,6.000,6.046618e+07,3.272727,3.656158e+15,1.653817e-08,1.0,1.0,3.272727,3.656158e+15
13783,3.0,0.0,9.0,7.0,0.0,13.0,7.0,-4.0,0.0,0.0,...,1601613.0,14.625,3.001242e+14,14.625000,3.001242e+14,5.336500e-09,1.0,1.0,42.545455,4.162222e+03
15054,1.0,0.0,5.0,4.0,0.0,10.0,8.0,-3.0,0.0,0.0,...,50.0,10.000,6.250000e+06,5.555556,3.906250e+13,8.000000e-06,1.0,1.0,10.000000,6.250000e+06
16405,1.0,0.0,4.0,3.0,0.0,12.0,10.0,-2.0,0.0,0.0,...,48.0,12.000,1.105920e+05,4.363636,6.492506e+16,4.340278e-04,1.0,1.0,8.000000,2.548040e+08


In [16]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Personality'], axis=1), df['Personality'], test_size=0.2, stratify=df['Personality'], random_state=47)

In [17]:
print(X_train.shape[0])
print(X_test.shape[0])

14819
3705


In [18]:
models = {
    'xgb': XGBClassifier,
    'lgbm': LGBMClassifier,
    'catboost': CatBoostClassifier,
    'rf': RandomForestClassifier,
    'et': ExtraTreesClassifier,
    'lr': LogisticRegression,  
    'mlp': MLPClassifier,     
    'gb': GradientBoostingClassifier,
    'hgb': HistGradientBoostingClassifier 
}
models_trained = {name: [] for name in models}

In [19]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_train_rf = pd.DataFrame(X_train_sc, columns=X_train.columns, index=X_train.index)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_rf, y_train)
importances = rf.feature_importances_
probs = importances / np.sum(importances)
feats = rf.feature_names_in_

/home/nicomore/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/sklearn/utils/extmath.py:1156: RuntimeWarning: overflow encountered in square
  temp **= 2
/home/nicomore/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction**2 / new_sample_count
/home/nicomore/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count
/home/nicomore/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:84: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


In [20]:
def objective(trial, model_name, X_train, y_train, X_valid, y_valid):
    early_stopping_rounds = 10

    if model_name == 'xgb':
        params = {
            'max_depth': trial.suggest_int('max_depth', 100, 300),
            'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 80, 120),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'eval_metric': 'logloss'
        }
        model = XGBClassifier(**params, early_stopping_rounds=early_stopping_rounds)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

    elif model_name == 'lgbm':
        params = {
            'max_depth': trial.suggest_int('max_depth', 100, 400),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 100, 120),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
            'min_child_weight': trial.suggest_float('min_child_weight', 0.0001, 0.01, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 50)
        }
        model = LGBMClassifier(**params, early_stopping_rounds=early_stopping_rounds)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])

    elif model_name == 'catboost':
        params = {
            'depth': trial.suggest_int('depth', 7, 16),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
            'iterations': trial.suggest_int('iterations', 100, 120),
            'verbose': 0
        }
        model = CatBoostClassifier(**params, early_stopping_rounds=early_stopping_rounds)
        model.fit(X_train, y_train, eval_set=(X_valid, y_valid))

    elif model_name == 'rf':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 80, 120),
            'max_depth': trial.suggest_int('max_depth', 100, 250)
        }
        model = RandomForestClassifier(**params)
        model.fit(X_train, y_train)

    elif model_name == 'et':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 60, 100),
            'max_depth': trial.suggest_int('max_depth', 1, 10)
        }
        model = ExtraTreesClassifier(**params)
        model.fit(X_train, y_train)

    elif model_name == 'lr':
        params = {
            'C': trial.suggest_float('C', 1e-4, 10.0, log=True),
            'max_iter': trial.suggest_int('max_iter', 100, 200)
        }
        model = LogisticRegression(**params)
        model.fit(X_train, y_train)

    elif model_name == 'mlp':
        width = trial.suggest_int('hidden_layer_width', 20, 120)
        depth = trial.suggest_int('hidden_layer_depth', 1, 15)
        params = {
            'hidden_layer_sizes': tuple([width] * depth),
            'activation': trial.suggest_categorical('activation', ['relu', 'tanh']),
            'alpha': trial.suggest_float('alpha', 1e-5, 1e-2, log=True),
            'learning_rate_init': trial.suggest_float('learning_rate_init', 1e-4, 1e-2, log=True),
            'max_iter': 1000,
            'early_stopping': True
        }
        model = MLPClassifier(**params)
        model.fit(X_train, y_train)

    elif model_name == 'gb':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 80, 120),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
            'max_depth': trial.suggest_int('max_depth', 3, 10)
        }
        model = GradientBoostingClassifier(**params)
        model.fit(X_train, y_train)

    elif model_name == 'hgb':
        params = {
            'max_iter': trial.suggest_int('max_iter', 100, 200),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
            'max_depth': trial.suggest_int('max_depth', 5, 20),
            'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 20, 60),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 5, 40)
        }
        model = HistGradientBoostingClassifier(**params)
        model.fit(X_train, y_train)

    try:
        calibrated_model = CalibratedClassifierCV(model, method='sigmoid', cv='prefit')
        calibrated_model.fit(X_valid, y_valid)
        preds = calibrated_model.predict(X_valid)
    except Exception as e:
        preds = model.predict(X_valid)

    return accuracy_score(y_valid, preds)


In [ ]:
trials = 10
folds = 5
size = 60

for i, name in enumerate(models):
    for j, (train_idx, valid_idx) in enumerate(StratifiedKFold(n_splits=folds, shuffle=True, random_state=47).split(X_train, y_train)):
        selected = np.random.default_rng(round(47*(i+1)/(j+1))).choice(feats, size=size, replace=False, p=probs)

        X_tr, X_valid = X_train[selected].iloc[train_idx], X_train[selected].iloc[valid_idx]
        X_tr = X_tr.clip(lower=X_tr.quantile(0.01), upper=X_tr.quantile(0.99), axis=1)

        scaler = StandardScaler()
        df_scaled_values = scaler.fit_transform(X_tr[selected])
        X_tr[selected] = pd.DataFrame(df_scaled_values, columns=selected, index=X_tr.index)

        df_scaled_values = scaler.transform(X_valid[selected])
        X_valid[selected] = pd.DataFrame(df_scaled_values, columns=selected, index=X_valid.index)

        y_tr, y_valid = y_train.iloc[train_idx], y_train.iloc[valid_idx]

        X_tr, y_tr = SMOTE(random_state=47).fit_resample(X_tr, y_tr)

        study = optuna.create_study(direction='maximize')
        obj = partial(objective, model_name=name, X_train=X_tr, y_train=y_tr, X_valid=X_valid, y_valid=y_valid)
        study.optimize(obj, n_trials=trials, show_progress_bar=False)

        best_params = study.best_params

        model_cls = models[name]

        if name == 'mlp':
            best_params['hidden_layer_sizes'] = tuple([best_params['hidden_layer_width']] * best_params['hidden_layer_depth'])
            del best_params['hidden_layer_width']
            del best_params['hidden_layer_depth']

        final_model = model_cls(**best_params)
        final_model.fit(X_tr, y_tr)

        models_trained[name].append((final_model, selected, scaler))

[I 2025-07-22 19:25:17,029] A new study created in memory with name: no-name-6ba67f73-e991-41ce-b0db-8ff019cf44a8
[I 2025-07-22 19:25:26,917] Trial 0 finished with value: 0.9676113360323887 and parameters: {'max_depth': 300, 'learning_rate': 0.004782896375780932, 'n_estimators': 98, 'subsample': 0.9524948454614519, 'colsample_bytree': 0.7825337669681691}. Best is trial 0 with value: 0.9676113360323887.
[I 2025-07-22 19:25:56,911] Trial 1 finished with value: 0.9676113360323887 and parameters: {'max_depth': 165, 'learning_rate': 0.00012396254981298104, 'n_estimators': 111, 'subsample': 0.7630575856804814, 'colsample_bytree': 0.743650923628053}. Best is trial 0 with value: 0.9676113360323887.
[I 2025-07-22 19:26:14,106] Trial 2 finished with value: 0.9652496626180836 and parameters: {'max_depth': 120, 'learning_rate': 0.029358014482471332, 'n_estimators': 117, 'subsample': 0.9046698327384363, 'colsample_bytree': 0.9217981922874057}. Best is trial 0 with value: 0.9676113360323887.
[I 2025

[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Info] Number of positive: 8767, number of negative: 8767
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001286 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14729
[LightGBM] [Info] Number of data points in the train set: 17534, number of used features: 60
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[112]	valid_0's binary_logloss: 0.143062
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM]

[I 2025-07-22 19:35:02,354] Trial 1 finished with value: 0.9672739541160594 and parameters: {'max_depth': 326, 'learning_rate': 0.03142283240817659, 'n_estimators': 120, 'subsample': 0.887275187727705, 'colsample_bytree': 0.7406560541736671, 'min_child_samples': 12, 'min_child_weight': 0.00012260521937005117, 'num_leaves': 39}. Best is trial 0 with value: 0.9682860998650472.
[I 2025-07-22 19:35:02,442] Trial 2 finished with value: 0.9672739541160594 and parameters: {'max_depth': 338, 'learning_rate': 0.06450999284061436, 'n_estimators': 115, 'subsample': 0.8784806271919186, 'colsample_bytree': 0.7795698580120688, 'min_child_samples': 22, 'min_child_weight': 0.0007203352588945199, 'num_leaves': 21}. Best is trial 0 with value: 0.9682860998650472.


Did not meet early stopping. Best iteration is:
[116]	valid_0's binary_logloss: 0.141939
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Info] Number of positive: 8767, number of negative: 8767
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001181 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14729
[LightGBM] [Info] Number of data points in the train set: 17534, number of used features: 60
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[68]	valid_0's binary_logloss: 0.14208
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_ro

[I 2025-07-22 19:35:02,559] Trial 3 finished with value: 0.9669365721997301 and parameters: {'max_depth': 318, 'learning_rate': 0.004866652785015947, 'n_estimators': 110, 'subsample': 0.6955114495708407, 'colsample_bytree': 0.7626321863087742, 'min_child_samples': 37, 'min_child_weight': 0.00024185660298851568, 'num_leaves': 28}. Best is trial 0 with value: 0.9682860998650472.
[I 2025-07-22 19:35:02,656] Trial 4 finished with value: 0.9676113360323887 and parameters: {'max_depth': 217, 'learning_rate': 0.0760860532961336, 'n_estimators': 106, 'subsample': 0.6403140034980229, 'colsample_bytree': 0.9831221573735629, 'min_child_samples': 41, 'min_child_weight': 0.00013425591537243704, 'num_leaves': 20}. Best is trial 0 with value: 0.9682860998650472.


...

[I 2025-07-22 20:00:59,967] Trial 0 finished with value: 0.9669254134323321 and parameters: {'depth': 15, 'learning_rate': 0.019845328470046244, 'iterations': 117}. Best is trial 0 with value: 0.9669254134323321.
[I 2025-07-22 20:01:00,688] Trial 1 finished with value: 0.9662504218697267 and parameters: {'depth': 7, 'learning_rate': 0.007220923242892805, 'iterations': 117}. Best is trial 0 with value: 0.9669254134323321.
[I 2025-07-22 20:01:01,692] Trial 2 finished with value: 0.9669254134323321 and parameters: {'depth': 8, 'learning_rate': 0.016068596723603137, 'iterations': 118}. Best is trial 0 with value: 0.9669254134323321.
[I 2025-07-22 20:02:23,107] Trial 3 finished with value: 0.9669254134323321 and parameters: {'depth': 15, 'learning_rate': 0.002847192400414913, 'iterations': 110}. Best is trial 0 with value: 0.9669254134323321.
[I 2025-07-22 20:02:40,649] Trial 4 finished with value: 0.9665879176510294 and parameters: {'depth': 13, 'learning_rate': 0.008224583697320998, 'iter

0:	learn: 0.6770984	total: 16.6ms	remaining: 1.97s
1:	learn: 0.6613219	total: 29.3ms	remaining: 1.73s
2:	learn: 0.6462456	total: 45ms	remaining: 1.75s
3:	learn: 0.6318268	total: 53.8ms	remaining: 1.56s
4:	learn: 0.6175358	total: 71.7ms	remaining: 1.65s
5:	learn: 0.6037434	total: 87.7ms	remaining: 1.67s
6:	learn: 0.5904706	total: 101ms	remaining: 1.63s
7:	learn: 0.5772059	total: 121ms	remaining: 1.69s
8:	learn: 0.5646737	total: 137ms	remaining: 1.69s
9:	learn: 0.5524763	total: 153ms	remaining: 1.68s
10:	learn: 0.5405231	total: 168ms	remaining: 1.67s
11:	learn: 0.5290242	total: 182ms	remaining: 1.64s
12:	learn: 0.5181946	total: 196ms	remaining: 1.61s
13:	learn: 0.5075930	total: 212ms	remaining: 1.61s
14:	learn: 0.4972697	total: 227ms	remaining: 1.59s
15:	learn: 0.4870973	total: 246ms	remaining: 1.6s
16:	learn: 0.4771575	total: 260ms	remaining: 1.58s
17:	learn: 0.4676524	total: 270ms	remaining: 1.53s
18:	learn: 0.4586033	total: 282ms	remaining: 1.5s
19:	learn: 0.4498646	total: 295ms	remai

[I 2025-07-22 20:02:55,842] A new study created in memory with name: no-name-5794071d-ba3b-4944-a1ae-bd04812bc3f3


113:	learn: 0.1760067	total: 1.56s	remaining: 82ms
114:	learn: 0.1754035	total: 1.57s	remaining: 68.2ms
115:	learn: 0.1747534	total: 1.58s	remaining: 54.7ms
116:	learn: 0.1741679	total: 1.6s	remaining: 40.9ms
117:	learn: 0.1735516	total: 1.61s	remaining: 27.3ms
118:	learn: 0.1729568	total: 1.62s	remaining: 13.6ms
119:	learn: 0.1723826	total: 1.63s	remaining: 0us


[I 2025-07-22 20:02:59,110] Trial 0 finished with value: 0.9612010796221323 and parameters: {'n_estimators': 101, 'max_depth': 130}. Best is trial 0 with value: 0.9612010796221323.
[I 2025-07-22 20:03:02,298] Trial 1 finished with value: 0.9618758434547908 and parameters: {'n_estimators': 98, 'max_depth': 229}. Best is trial 1 with value: 0.9618758434547908.
[I 2025-07-22 20:03:05,558] Trial 2 finished with value: 0.9601889338731444 and parameters: {'n_estimators': 100, 'max_depth': 220}. Best is trial 1 with value: 0.9618758434547908.
[I 2025-07-22 20:03:08,218] Trial 3 finished with value: 0.9612010796221323 and parameters: {'n_estimators': 81, 'max_depth': 133}. Best is trial 1 with value: 0.9618758434547908.
[I 2025-07-22 20:03:12,046] Trial 4 finished with value: 0.9608636977058029 and parameters: {'n_estimators': 119, 'max_depth': 184}. Best is trial 1 with value: 0.9618758434547908.
[I 2025-07-22 20:03:14,958] Trial 5 finished with value: 0.9612010796221323 and parameters: {'n_e

In [ ]:
feature_counter = Counter()

for model_type in models_trained:
    for model in models_trained[model_type]:
        feature_counter.update(list(model[1]))

feature_counter

Counter({'Going_outside_*_Media_Dependence': 45,
         'Social_event_attendance_*_Media_Dependence': 45,
         'Exp_Reclusion': 45,
         'Friends_circle_size_*_Media_Dependence': 45,
         'Reclusion_/_Post_frequency': 45,
         'Media_Dependence_*_Social_Volume': 45,
         'Social_event_attendance_*_Post_frequency': 45,
         'Post_frequency_*_Social_Volume': 45,
         'Time_spent_Alone_/_Post_frequency': 45,
         'Stage_fear_+_Drained_after_socializing': 44,
         'Log1p_Reclusion': 43,
         'Social_Volume_/_Time_spent_Alone': 43,
         'Going_outside_*_Social_Volume': 43,
         'Going_outside_**_Social_event_attendance': 43,
         'Sqrt_Reclusion': 42,
         'Time_spent_Alone_/_Social_Volume': 42,
         'Social_event_attendance_+_Media_Dependence': 42,
         'Time_spent_Alone_/_Going_outside': 41,
         'Social_Drainage_+_Actual_Fear_Exposure': 41,
         'Time_spent_Alone_-_Social_event_attendance': 41,
         'Going_outs

In [ ]:
XT = X_test.copy()

preds = pd.DataFrame(index=X_test.index)
for model_type in models_trained:
    for i, model in enumerate(models_trained[model_type]):
        model, columns, scaler = model
        X_test_sc = scaler.transform(X_test[columns])
        X_test[columns] = pd.DataFrame(X_test_sc, columns=columns, index=X_test.index)
        preds[model_type + '_' + str(i)] = model.predict_proba(X_test[columns])[:, 1]

In [ ]:
preds.corr()

,xgb_0,xgb_1,xgb_2,xgb_3,xgb_4,lgbm_0,lgbm_1,lgbm_2,lgbm_3,lgbm_4,...,gb_0,gb_1,gb_2,gb_3,gb_4,hgb_0,hgb_1,hgb_2,hgb_3,hgb_4
xgb_0,1.000000,0.945442,0.894016,0.879393,0.872739,0.912050,0.821600,0.673942,0.862313,0.945085,...,0.802830,0.853112,0.752593,0.878930,0.462791,0.851145,0.805057,0.852323,0.787943,0.893668
xgb_1,0.945442,1.000000,0.872454,0.857925,0.833139,0.884406,0.783958,0.662338,0.832704,0.913030,...,0.765023,0.819628,0.722274,0.867435,0.436872,0.816616,0.766074,0.810533,0.756276,0.853234
xgb_2,0.894016,0.872454,1.000000,0.931458,0.913449,0.936576,0.886047,0.753261,0.925072,0.931930,...,0.861533,0.890231,0.839760,0.924739,0.485463,0.913144,0.865062,0.881350,0.867047,0.918426
xgb_3,0.879393,0.857925,0.931458,1.000000,0.960123,0.947213,0.947302,0.825121,0.966729,0.923026,...,0.929199,0.939555,0.909225,0.935697,0.478407,0.947044,0.930039,0.938299,0.935635,0.922741
xgb_4,0.872739,0.833139,0.913449,0.960123,1.000000,0.956123,0.983985,0.841841,0.980533,0.925161,...,0.974056,0.981825,0.938481,0.928812,0.476397,0.966788,0.971202,0.976772,0.951352,0.944996
lgbm_0,0.912050,0.884406,0.936576,0.947213,0.956123,1.000000,0.935759,0.799517,0.970400,0.985537,...,0.908991,0.955486,0.879544,0.970206,0.506099,0.942732,0.896253,0.915962,0.901268,0.933256
lgbm_1,0.821600,0.783958,0.886047,0.947302,0.983985,0.935759,1.000000,0.863584,0.987021,0.888606,...,0.988358,0.977388,0.969522,0.918591,0.464233,0.976915,0.983507,0.983281,0.973547,0.924041
lgbm_2,0.673942,0.662338,0.753261,0.825121,0.841841,0.799517,0.863584,1.000000,0.846613,0.748727,...,0.857593,0.843086,0.842921,0.786686,0.520236,0.841979,0.847164,0.837141,0.844610,0.780086
lgbm_3,0.862313,0.832704,0.925072,0.966729,0.980533,0.970400,0.987021,0.846613,1.000000,0.933752,...,0.963505,0.968484,0.954820,0.957377,0.484618,0.982224,0.957004,0.964641,0.967501,0.933936
lgbm_4,0.945085,0.913030,0.931930,0.923026,0.925161,0.985537,0.888606,0.748727,0.933752,1.000000,...,0.860568,0.920792,0.823402,0.953685,0.509440,0.908300,0.851833,0.881922,0.850422,0.919886


In [ ]:
nnscaler = StandardScaler()
preds = nnscaler.fit_transform(preds)

In [ ]:
class KillWhenLowAcc(tf.keras.callbacks.Callback):
    def __init__(self, minaccuracy, epoch):
        super(KillWhenLowAcc, self).__init__()
        self.minaccuracy = minaccuracy
        self.epoch = epoch
        self.maxaccuracyyet = 0.0
        self.epochindex = 0

    def on_epoch_end(self, epoch, logs=None):

        accuracy = logs.get('val_accuracy')

        if accuracy > self.maxaccuracyyet:
            self.maxaccuracyyet = accuracy

        if self.epochindex < len(self.epoch) and epoch == self.epoch[self.epochindex]:

            if self.maxaccuracyyet < self.minaccuracy[self.epochindex]:
                self.model.stop_training = True

            self.epochindex += 1


class HyperNet(kt.HyperModel):
    def __init__(self, **kwargs):
        self.ninputs = kwargs.get('ninputs')
        self.noutputs = kwargs.get('noutputs')
        super().__init__()

    def build(self, hp):
        nlength = hp.Int('nlength', min_value=1, max_value=6, step=1)
        nwidth = hp.Int('nwidth', min_value=50, max_value=150, step=1)

        input_ = tf.keras.layers.Input(shape=(self.ninputs,))

        for n in range(0, nlength):
            bn = tf.keras.layers.BatchNormalization()
            midlayer = tf.keras.layers.Dense(nwidth, activation='leaky_relu', kernel_initializer='he_normal', use_bias=False)
            do = tf.keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.6, step=0.01))

            if n == 0:
                mid = do(midlayer(bn(input_)))
            elif n != nlength-1:
                mid = do(midlayer(bn(mid)))
            else:
                mid = midlayer(bn(mid))

        bn = tf.keras.layers.BatchNormalization()
        lastdo = tf.keras.layers.Dropout(rate=hp.Float('lastdropout_rate', min_value=0.0, max_value=0.6, step=0.01))
        outputlayer = tf.keras.layers.Dense(1, activation='sigmoid')

        output = outputlayer(lastdo(bn(mid)))

        model = tf.keras.Model(inputs=[input_], outputs=[output])
        opt = tf.keras.optimizers.Nadam(learning_rate=hp.Float('learning_rate', min_value=0.0001, max_value=0.1, step=0.001))

        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

        return model

In [ ]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
else:
    print("GPU is not available.")

GPU is available.


I0000 00:00:1753217152.411298   16274 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1753217152.549609   16274 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1753217152.549657   16274 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [ ]:
X_trainnn, X_tnn, y_trainnn, y_tnn = train_test_split(preds, y_test, test_size=0.2, stratify=y_test, random_state=47)

class_weights = dict(enumerate(class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_trainnn), y=y_trainnn)))

tuner = kt.BayesianOptimization(hypermodel=HyperNet(ninputs=X_trainnn.shape[1], noutputs=len(np.unique(y_trainnn))),
                                objective='val_accuracy',
                                max_trials=trials,
                                max_retries_per_trial=0,
                                max_consecutive_failed_trials=trials,
                                overwrite=True,
                                seed=47)

rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, min_lr=0.00001)
es = tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True)
kwa = KillWhenLowAcc(minaccuracy=[0.5, 0.8, 0.9], epoch=[2, 14, 29])

X_trainnn, X_tnn, y_trainnn, y_tnn = X_trainnn, X_tnn, y_trainnn.to_numpy(), y_tnn.to_numpy()
tuner.search(X_trainnn, y_trainnn, batch_size=64, validation_data=(X_tnn, y_tnn), class_weight=class_weights, callbacks=[es, rlr, kwa], epochs=1000)
bestmodel = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters()[0]


Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.9635627269744873

Best val_accuracy So Far: 0.9635627269744873
Total elapsed time: 00h 00m 24s


/home/nicomore/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 29 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
ridge = RidgeClassifier()
ridge.fit(preds, y_test)

lr = LogisticRegression()
lr.fit(preds, y_test)

LogisticRegression()

In [ ]:
df_sub['id'] = df_sub['id'].astype(int)
df_sub = df_sub.set_index('id')

In [ ]:
df_sub = df_sub.replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:
fpreds = pd.DataFrame(index=df_sub.index)
for model_type in models_trained:
    for i, model in enumerate(models_trained[model_type]):
        model, columns, scaler = model
        df_sub_sc = scaler.transform(df_sub[columns])
        df_sub[columns] = pd.DataFrame(df_sub_sc, columns=columns, index=df_sub.index)
        fpreds[model_type + '_' + str(i)] = model.predict_proba(df_sub[columns])[:, 1]

In [ ]:
fpreds_id = pd.DataFrame(fpreds.reset_index()['id'])
rpreds_id = pd.DataFrame(fpreds.reset_index()['id'])
lpreds_id = pd.DataFrame(fpreds.reset_index()['id'])
oldfpreds = fpreds.reset_index(drop=True)
fpreds = pd.DataFrame(nnscaler.transform(fpreds), columns=fpreds.columns)

In [ ]:
fpreds_id['pred'] = bestmodel.predict(fpreds)

193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
naive = fpreds_id.copy()
naive['pred'] = oldfpreds.sum(axis=1)

In [ ]:
Xpreds = pd.DataFrame(index=X_train.index)
for model_type in models_trained:
    for i, model in enumerate(models_trained[model_type]):
        model, columns, scaler = model
        X_train_sc = scaler.transform(X_train[columns])
        X_train[columns] = pd.DataFrame(X_train_sc, columns=columns, index=X_train.index)
        Xpreds[model_type + '_' + str(i)] = model.predict_proba(X_train[columns])[:, 1]

In [ ]:
best_score = 0
po = bestmodel.predict(Xpreds)

for t in np.arange(0.80, 0.10, -0.01):
    p = (po >= t).astype(int)
    score = accuracy_score(y_train, p)
    if score > best_score:
        best_score = score
        best_threshold_nn = t

print(best_threshold_nn)
print(best_score)

464/464 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
0.8
0.26047641541264593


In [ ]:
best_score = 0

for t in np.arange(100.0, 1.0, -0.01):
    score = accuracy_score(y_train, np.where(Xpreds.sum(axis=1) >= t, 1, 0))
    if score > best_score:
        best_score = score
        best_threshold_naive = t

print(best_threshold_naive)
print(best_score)

19.359999999958745
0.967811593224914


In [ ]:
best_score = 0
po = ridge.predict(Xpreds)

for t in np.arange(0.80, 0.10, -0.05):
    p = (po >= t).astype(int)
    score = accuracy_score(y_train, p)
    if score > best_score:
        best_score = score
        best_threshold_ridge = t

print(best_threshold_ridge)
print(best_score)

0.8
0.9661245698090289


/home/nicomore/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RidgeClassifier was fitted without feature names
  warnings.warn(


In [ ]:
best_score = 0
po = lr.predict_proba(Xpreds)[:,1]

for t in np.arange(0.80, 0.10, -0.05):
    p = (po >= t).astype(int)
    score = accuracy_score(y_train, p)
    if score > best_score:
        best_score = score
        best_threshold_lr = t

print(best_threshold_lr)
print(best_score)

0.44999999999999973
0.8779944665631959


/home/nicomore/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [ ]:
naive['pred'] = np.where(naive['pred'] > best_threshold_naive, 'Introvert', 'Extrovert')

In [ ]:
naive.to_csv('submission_naive.csv', index=False)

In [ ]:
fpreds_id

,id,pred
0,21760,1.000000
1,19687,0.000031
2,23291,0.000051
3,19646,1.000000
4,24142,1.000000
...,...,...
6170,20054,1.000000
6171,24319,0.000109
6172,22817,0.000030
6173,21678,1.000000


In [ ]:
fpreds_id['pred'] = np.where(fpreds_id['pred'] >= best_threshold_nn, 'Introvert', 'Extrovert')

In [ ]:
fpreds_id.to_csv('submission.csv', index=False)

In [ ]:
rpreds_id['pred'] = ridge.predict(fpreds)
rpreds_id['pred'] = np.where(rpreds_id['pred'] >= best_threshold_ridge, 'Introvert', 'Extrovert')

/home/nicomore/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RidgeClassifier was fitted without feature names
  warnings.warn(


In [ ]:
rpreds_id.to_csv('submission_ridge.csv', index=False)

In [ ]:
lpreds_id['pred'] = ridge.predict(fpreds)
lpreds_id['pred'] = np.where(lpreds_id['pred'] >= best_threshold_ridge, 'Introvert', 'Extrovert')

/home/nicomore/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RidgeClassifier was fitted without feature names
  warnings.warn(


In [ ]:
lpreds_id.to_csv('submission_lr.csv', index=False)